In [1]:
using Plots
using LinearAlgebra
using Jacobi
using Test
using SparseArrays

In [2]:
function TrilinearMap(Coord_E, xhat, yhat, zhat)
    """
        local numbering of Hex
    
        5--------7           z
      / |      / |            |
    6 --|---- 8  |            |
    |   |     |  |            /----y
    |   |     |  |           /
    |   1 -------3         x
    | /       | /
    2 --------4
    """
    m = length(xhat)
    N1 = @. 0.125*(1-xhat)*(1-yhat)*(1-zhat)
    N2 = @. 0.125*(1+xhat)*(1-yhat)*(1-zhat)
    N3 = @. 0.125*(1-xhat)*(1+yhat)*(1-zhat)
    N4 = @. 0.125*(1+xhat)*(1+yhat)*(1-zhat)
    N5 = @. 0.125*(1-xhat)*(1-yhat)*(1+zhat)
    N6 = @. 0.125*(1+xhat)*(1-yhat)*(1+zhat)
    N7 = @. 0.125*(1-xhat)*(1+yhat)*(1+zhat)
    N8 = @. 0.125*(1+xhat)*(1+yhat)*(1+zhat)
    N = [N1 N2 N3 N4 N5 N6 N7 N8]
    X = N * Coord_E
    # X(3,m), 1st row x, 2nd row y, 3rd row z
    X = X'
    # derivatives of shape functions with respect to xhat
    dN1_dxhat = @. -0.125*(1-yhat)*(1-zhat)
    dN2_dxhat = -dN1_dxhat
    dN3_dxhat = @. -0.125*(1+yhat)*(1-zhat)
    dN4_dxhat = -dN3_dxhat
    dN5_dxhat = @. -0.125*(1-yhat)*(1+zhat)
    dN6_dxhat = -dN5_dxhat
    dN7_dxhat = @. -0.125*(1+yhat)*(1+zhat)
    dN8_dxhat = -dN7_dxhat

    # derivatives of shape functions with respect to yhat
    dN1_dyhat = @. -0.125*(1-xhat)*(1-zhat)
    dN2_dyhat = @. -0.125*(1+xhat)*(1-zhat)
    dN3_dyhat = -dN1_dyhat
    dN4_dyhat = -dN2_dyhat
    dN5_dyhat = @. -0.125*(1-xhat)*(1+zhat)
    dN6_dyhat = @. -0.125*(1+xhat)*(1+zhat)
    dN7_dyhat = -dN5_dyhat
    dN8_dyhat = -dN6_dyhat

    # derivatives of shape functions with respect to zhat
    dN1_dzhat = @. -0.125*(1-xhat)*(1-yhat)
    dN2_dzhat = @. -0.125*(1+xhat)*(1-yhat)
    dN3_dzhat = @. -0.125*(1-xhat)*(1+yhat)
    dN4_dzhat = @. -0.125*(1+xhat)*(1+yhat)
    dN5_dzhat = -dN1_dzhat
    dN6_dzhat = -dN2_dzhat
    dN7_dzhat = -dN3_dzhat
    dN8_dzhat = -dN4_dzhat
    
    # gradient of N, [dN/dxhat; dN/dyhat; dN/dzhat]
    GradN = zeros(3,m,8)
    GradN[1,:,:] = [dN1_dxhat dN2_dxhat dN3_dxhat dN4_dxhat dN5_dxhat dN6_dxhat dN7_dxhat dN8_dxhat]
    GradN[2,:,:] = [dN1_dyhat dN2_dyhat dN3_dyhat dN4_dyhat dN5_dyhat dN6_dyhat dN7_dyhat dN8_dyhat]
    GradN[3,:,:] = [dN1_dzhat dN2_dzhat dN3_dzhat dN4_dzhat dN5_dzhat dN6_dzhat dN7_dzhat dN8_dzhat]
    
    # JT = [[dx/dxhat, dy/dxhat, dz/dxhat],
    #       [dx/dyhat, dy/dyhat, dz/dyhat],
    #       [dx/dzhat, dy/dzhat, dz/dzhat]] (3m x 3)

    JTxhat = GradN[1,:,:] * Coord_E
    dxdxhat = JTxhat[:,1]
    dydxhat = JTxhat[:,2]
    dzdxhat = JTxhat[:,3]
    JTyhat = GradN[2,:,:] * Coord_E
    dxdyhat = JTyhat[:,1]
    dydyhat = JTyhat[:,2]
    dzdyhat = JTyhat[:,3]
    JTzhat = GradN[3,:,:] * Coord_E
    dxdzhat = JTzhat[:,1]
    dydzhat = JTzhat[:,2]
    dzdzhat = JTzhat[:,3]
    # compute det
    j1 = @. (dxdxhat*dydyhat*dzdzhat + dxdyhat*dydzhat*dzdzhat + dxdzhat*dydxhat*dzdyhat)
    j2 = @. (dxdxhat*dydzhat*dzdyhat + dxdyhat*dydxhat*dzdzhat + dxdzhat*dydyhat*dzdxhat)
    detJ = @. j1 - j2
    
    J = zeros(3,m,3)
    J[1,:,:] = [dxdxhat dxdyhat dxdzhat]
    J[2,:,:] = [dydxhat dydyhat dydzhat]
    J[3,:,:] = [dzdxhat dzdyhat dzdzhat]
    
    return X, J, detJ
end

TrilinearMap (generic function with 1 method)

In [3]:
function GetNormal(Coord_E, xhat, yhat, zhat, face)
    """
    Input:
    Coord_E: coordinate of physical element E as a 8x3 matrix
    xhat, yhat, zhat: are defined on the reference element Ehat
    and should be given as a vector (xhat=[1] or xhat =[1;1],...)
    face: that you want the normal
    Note the face and xhat, yhat, zhat should be consistent.
    
    Return:
    n: of size(3,m)
    le: length of n
    
    Based on following numbering:
    left:     nodes 1,2,5,6 at yhat = -1
    right:    nodes 3,4,7,8 at yhat =  1
    bottom:   nodes 1,2,3,4 at zhat = -1
    top:      nodes 5,6,7,8 at zhat =  1
    front:    nodes 2,4,6,8 at xhat =  1
    back:     nodes 1,3,5,7 at xhat = -1
    
    local numbering of Hex
    
        5--------7           z
      / |      / |            |
    6 --|---- 8  |            |
    |   |     |  |            /----y
    |   |     |  |           /
    |   1 -------3         x
    | /       | /
    2 --------4
    """
    X, J, detJ = TrilinearMap(Coord_E, xhat, yhat, zhat)

    dxdxhat = J[1,:,1]
    dxdyhat = J[1,:,2]
    dxdzhat = J[1,:,3]
    
    dydxhat = J[2,:,1]
    dydyhat = J[2,:,2]
    dydzhat = J[2,:,3]
    
    dzdxhat = J[3,:,1]
    dzdyhat = J[3,:,2]
    dzdzhat = J[3,:,3]

    m = length(xhat)
    
    if face == "left" && yhat == -ones(m)
        
        n1 = @. dydxhat*dzdzhat - dzdxhat*dydzhat
        n2 = @. dzdxhat*dxdzhat - dxdxhat*dzdzhat
        n3 = @. dxdxhat*dydzhat - dydxhat*dxdzhat
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = zeros(3,m)
        n[1,:] = n1 ./ leng
        n[2,:] = n2 ./ leng
        n[3,:] = n3 ./ leng 
    
    elseif face == "right" && yhat == ones(m)
        
        n1 = @. dzdxhat*dydzhat - dydxhat*dzdzhat
        n2 = @. dxdxhat*dzdzhat - dzdxhat*dxdzhat
        n3 = @. dydxhat*dxdzhat - dxdxhat*dydzhat 
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = zeros(3,m)
        n[1,:] = n1 ./ leng
        n[2,:] = n2 ./ leng
        n[3,:] = n3 ./ leng
        
    elseif face == "bottom" && zhat == -ones(m)
        
        n1 = @. dzdxhat*dydyhat - dydxhat*dzdyhat
        n2 = @. dxdxhat*dzdyhat - dzdxhat*dxdyhat
        n3 = @. dydxhat*dxdyhat - dxdxhat*dydyhat 
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = zeros(3,m)
        n[1,:] = n1 ./ leng
        n[2,:] = n2 ./ leng
        n[3,:] = n3 ./ leng
        
    elseif face == "top" && zhat == ones(m)
        
        n1 = @. dydxhat*dzdyhat - dzdxhat*dydyhat 
        n2 = @. dzdxhat*dxdyhat - dxdxhat*dzdyhat 
        n3 = @. dxdxhat*dydyhat - dydxhat*dxdyhat 
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = zeros(3,m)
        n[1,:] = n1 ./ leng
        n[2,:] = n2 ./ leng
        n[3,:] = n3 ./ leng
        
    elseif face == "front" && xhat == ones(m)
        
        n1 = @. dydyhat*dzdzhat - dzdyhat*dydzhat
        n2 = @. dzdyhat*dxdzhat - dxdyhat*dzdzhat
        n3 = @. dxdyhat*dydzhat - dydyhat*dxdzhat
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = zeros(3,m)
        n[1,:] = n1 ./ leng
        n[2,:] = n2 ./ leng
        n[3,:] = n3 ./ leng
        
    elseif face == "back" && xhat == -ones(m)
        
        n1 = @. dzdyhat*dydzhat - dydyhat*dzdzhat 
        n2 = @. dxdyhat*dzdzhat - dzdyhat*dxdzhat
        n3 = @. dydyhat*dxdzhat - dxdyhat*dydzhat 
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = zeros(3,m)
        n[1,:] = n1 ./ leng
        n[2,:] = n2 ./ leng
        n[3,:] = n3 ./ leng 
    else
        error("face is not defined")
    
    end

    return n, leng
end

GetNormal (generic function with 1 method)

In [4]:
function GetBasis(xhat, yhat, zhat)

    m = length(xhat)
    Bx = zeros(m,24)
    By = zeros(m,24)
    Bz = zeros(m,24)
    Bx[:,1] = @. -0.125 + 0.125*xhat + 0.125*yhat + 0.125*zhat + -0.125*xhat*yhat + -0.125*xhat*yhat + -0.125*yhat*zhat + 0.125*xhat*yhat*zhat;
    By[:,1] = @. -0.0625 + 0.125*yhat*zhat + 0.0625*yhat*yhat + -0.125*yhat*zhat;
    Bz[:,1] = @. -0.0625 + 0.0625*yhat + 0.0625*zhat*zhat + -0.0625*yhat*zhat*zhat;
    Bx[:,2] = @. -0.125 + 0.125*xhat + -0.125*yhat + 0.125*zhat + -0.125*xhat*yhat + 0.125*xhat*yhat + 0.125*yhat*zhat + -0.125*xhat*yhat*zhat;
    By[:,2] = @. 0.0625 + 0.125*yhat*zhat + -0.0625*yhat*yhat + -0.125*yhat*zhat;
    Bz[:,2] = @. -0.0625 + -0.0625*yhat + 0.0625*zhat*zhat + 0.0625*yhat*zhat*zhat;
    Bx[:,3] = @. -0.125 + 0.125*xhat + 0.125*yhat + -0.125*zhat + 0.125*xhat*yhat + -0.125*xhat*yhat + 0.125*yhat*zhat + -0.125*xhat*yhat*zhat;
    By[:,3] = @. -0.0625 + -0.125*yhat*zhat + 0.0625*yhat*yhat + 0.125*yhat*zhat;
    Bz[:,3] = @. 0.0625 + -0.0625*yhat + -0.0625*zhat*zhat + 0.0625*yhat*zhat*zhat;
    Bx[:,4] = @. -0.125 + 0.125*xhat + -0.125*yhat + -0.125*zhat + 0.125*xhat*yhat + 0.125*xhat*yhat + -0.125*yhat*zhat + 0.125*xhat*yhat*zhat;
    By[:,4] = @. 0.0625 + -0.125*yhat*zhat + -0.0625*yhat*yhat + 0.125*yhat*zhat;
    Bz[:,4] = @. 0.0625 + 0.0625*yhat + -0.0625*zhat*zhat + -0.0625*yhat*zhat*zhat;
    Bx[:,5] = @. 0.125 + 0.125*xhat + -0.125*yhat + -0.125*zhat + -0.125*xhat*yhat + -0.125*xhat*yhat + 0.125*yhat*zhat + 0.125*xhat*yhat*zhat;
    By[:,5] = @. -0.0625 + 0.125*yhat*zhat + 0.0625*yhat*yhat + -0.125*yhat*zhat;
    Bz[:,5] = @. -0.0625 + 0.0625*yhat + 0.0625*zhat*zhat + -0.0625*yhat*zhat*zhat;
    Bx[:,6] = @. 0.125 + 0.125*xhat + 0.125*yhat + -0.125*zhat + -0.125*xhat*yhat + 0.125*xhat*yhat + -0.125*yhat*zhat + -0.125*xhat*yhat*zhat;
    By[:,6] = @. 0.0625 + 0.125*yhat*zhat + -0.0625*yhat*yhat + -0.125*yhat*zhat;
    Bz[:,6] = @. -0.0625 + -0.0625*yhat + 0.0625*zhat*zhat + 0.0625*yhat*zhat*zhat;
    Bx[:,7] = @. 0.125 + 0.125*xhat + -0.125*yhat + 0.125*zhat + 0.125*xhat*yhat + -0.125*xhat*yhat + -0.125*yhat*zhat + -0.125*xhat*yhat*zhat;
    By[:,7] = @. -0.0625 + -0.125*yhat*zhat + 0.0625*yhat*yhat + 0.125*yhat*zhat;
    Bz[:,7] = @. 0.0625 + -0.0625*yhat + -0.0625*zhat*zhat + 0.0625*yhat*zhat*zhat;
    Bx[:,8] = @. 0.125 + 0.125*xhat + 0.125*yhat + 0.125*zhat + 0.125*xhat*yhat + 0.125*xhat*yhat + 0.125*yhat*zhat + 0.125*xhat*yhat*zhat;
    By[:,8] = @. 0.0625 + -0.125*yhat*zhat + -0.0625*yhat*yhat + 0.125*yhat*zhat;
    Bz[:,8] = @. 0.0625 + 0.0625*yhat + -0.0625*zhat*zhat + -0.0625*yhat*zhat*zhat;
    Bx[:,9] = @. -0.0625 + -0.125*xhat*yhat + 0.125*xhat*yhat + 0.0625*xhat*xhat;
    By[:,9] = @. -0.0625 + 0.0625*xhat + 0.0625*yhat*yhat + -0.0625*xhat*yhat*yhat;
    Bz[:,9] = @. 0.125 + -0.125*xhat + -0.125*yhat + 0.125*zhat + 0.125*xhat*yhat + 0.125*xhat*yhat*zhat + -0.125*yhat*zhat + -0.125*xhat*zhat;
    Bx[:,10] = @. 0.0625 + -0.125*xhat*yhat + 0.125*xhat*yhat + -0.0625*xhat*xhat;
    By[:,10] = @. -0.0625 + -0.0625*xhat + 0.0625*yhat*yhat + 0.0625*xhat*yhat*yhat;
    Bz[:,10] = @. 0.125 + 0.125*xhat + -0.125*yhat + 0.125*zhat + -0.125*xhat*yhat + -0.125*xhat*yhat*zhat + -0.125*yhat*zhat + 0.125*xhat*zhat;
    Bx[:,11] = @. -0.0625 + 0.125*xhat*yhat + -0.125*xhat*yhat + 0.0625*xhat*xhat;
    By[:,11] = @. 0.0625 + -0.0625*xhat + -0.0625*yhat*yhat + 0.0625*xhat*yhat*yhat;
    Bz[:,11] = @. 0.125 + -0.125*xhat + 0.125*yhat + 0.125*zhat + -0.125*xhat*yhat + -0.125*xhat*yhat*zhat + 0.125*yhat*zhat + -0.125*xhat*zhat;
    Bx[:,12] = @. 0.0625 + 0.125*xhat*yhat + -0.125*xhat*yhat + -0.0625*xhat*xhat;
    By[:,12] = @. 0.0625 + 0.0625*xhat + -0.0625*yhat*yhat + -0.0625*xhat*yhat*yhat;
    Bz[:,12] = @. 0.125 + 0.125*xhat + 0.125*yhat + 0.125*zhat + 0.125*xhat*yhat + 0.125*xhat*yhat*zhat + 0.125*yhat*zhat + 0.125*xhat*zhat;
    Bx[:,13] = @. 0.0625 + -0.0625*zhat + 0.0625*xhat*xhat*zhat + -0.0625*xhat*xhat;
    By[:,13] = @. -0.125 + -0.125*xhat + 0.125*yhat + 0.125*zhat + 0.125*xhat*zhat + -0.125*xhat*yhat*zhat + 0.125*xhat*yhat + -0.125*yhat*zhat;
    Bz[:,13] = @. -0.0625 + -0.125*xhat*zhat + 0.0625*zhat*zhat + 0.125*xhat*zhat;
    Bx[:,14] = @. -0.0625 + 0.0625*zhat + -0.0625*xhat*xhat*zhat + 0.0625*xhat*xhat;
    By[:,14] = @. -0.125 + 0.125*xhat + 0.125*yhat + 0.125*zhat + -0.125*xhat*zhat + 0.125*xhat*yhat*zhat + -0.125*xhat*yhat + -0.125*yhat*zhat;
    Bz[:,14] = @. -0.0625 + 0.125*xhat*zhat + 0.0625*zhat*zhat + -0.125*xhat*zhat;
    Bx[:,15] = @. 0.0625 + 0.0625*zhat + -0.0625*xhat*xhat*zhat + -0.0625*xhat*xhat;
    By[:,15] = @. -0.125 + -0.125*xhat + 0.125*yhat + -0.125*zhat + -0.125*xhat*zhat + 0.125*xhat*yhat*zhat + 0.125*xhat*yhat + 0.125*yhat*zhat;
    Bz[:,15] = @. 0.0625 + -0.125*xhat*zhat + -0.0625*zhat*zhat + 0.125*xhat*zhat;
    Bx[:,16] = @. -0.0625 + -0.0625*zhat + 0.0625*xhat*xhat*zhat + 0.0625*xhat*xhat;
    By[:,16] = @. -0.125 + 0.125*xhat + 0.125*yhat + -0.125*zhat + 0.125*xhat*zhat + -0.125*xhat*yhat*zhat + -0.125*xhat*yhat + 0.125*yhat*zhat;
    Bz[:,16] = @. 0.0625 + 0.125*xhat*zhat + -0.0625*zhat*zhat + -0.125*xhat*zhat;
    Bx[:,17] = @. -0.0625 + -0.125*xhat*yhat + 0.125*xhat*yhat + 0.0625*xhat*xhat;
    By[:,17] = @. -0.0625 + 0.0625*xhat + 0.0625*yhat*yhat + -0.0625*xhat*yhat*yhat;
    Bz[:,17] = @. -0.125 + 0.125*xhat + 0.125*yhat + 0.125*zhat + -0.125*xhat*yhat + 0.125*xhat*yhat*zhat + -0.125*yhat*zhat + -0.125*xhat*zhat;
    Bx[:,18] = @. 0.0625 + -0.125*xhat*yhat + 0.125*xhat*yhat + -0.0625*xhat*xhat;
    By[:,18] = @. -0.0625 + -0.0625*xhat + 0.0625*yhat*yhat + 0.0625*xhat*yhat*yhat;
    Bz[:,18] = @. -0.125 + -0.125*xhat + 0.125*yhat + 0.125*zhat + 0.125*xhat*yhat + -0.125*xhat*yhat*zhat + -0.125*yhat*zhat + 0.125*xhat*zhat;
    Bx[:,19] = @. -0.0625 + 0.125*xhat*yhat + -0.125*xhat*yhat + 0.0625*xhat*xhat;
    By[:,19] = @. 0.0625 + -0.0625*xhat + -0.0625*yhat*yhat + 0.0625*xhat*yhat*yhat;
    Bz[:,19] = @. -0.125 + 0.125*xhat + -0.125*yhat + 0.125*zhat + 0.125*xhat*yhat + -0.125*xhat*yhat*zhat + 0.125*yhat*zhat + -0.125*xhat*zhat;
    Bx[:,20] = @. 0.0625 + 0.125*xhat*yhat + -0.125*xhat*yhat + -0.0625*xhat*xhat;
    By[:,20] = @. 0.0625 + 0.0625*xhat + -0.0625*yhat*yhat + -0.0625*xhat*yhat*yhat;
    Bz[:,20] = @. -0.125 + -0.125*xhat + -0.125*yhat + 0.125*zhat + -0.125*xhat*yhat + 0.125*xhat*yhat*zhat + 0.125*yhat*zhat + 0.125*xhat*zhat;
    Bx[:,21] = @. 0.0625 + -0.0625*zhat + 0.0625*xhat*xhat*zhat + -0.0625*xhat*xhat;
    By[:,21] = @. 0.125 + 0.125*xhat + 0.125*yhat + -0.125*zhat + -0.125*xhat*zhat + -0.125*xhat*yhat*zhat + 0.125*xhat*yhat + -0.125*yhat*zhat;
    Bz[:,21] = @. -0.0625 + -0.125*xhat*zhat + 0.0625*zhat*zhat + 0.125*xhat*zhat;
    Bx[:,22] = @. -0.0625 + 0.0625*zhat + -0.0625*xhat*xhat*zhat + 0.0625*xhat*xhat;
    By[:,22] = @. 0.125 + -0.125*xhat + 0.125*yhat + -0.125*zhat + 0.125*xhat*zhat + 0.125*xhat*yhat*zhat + -0.125*xhat*yhat + -0.125*yhat*zhat;
    Bz[:,22] = @. -0.0625 + 0.125*xhat*zhat + 0.0625*zhat*zhat + -0.125*xhat*zhat;
    Bx[:,23] = @. 0.0625 + 0.0625*zhat + -0.0625*xhat*xhat*zhat + -0.0625*xhat*xhat;
    By[:,23] = @. 0.125 + 0.125*xhat + 0.125*yhat + 0.125*zhat + 0.125*xhat*zhat + 0.125*xhat*yhat*zhat + 0.125*xhat*yhat + 0.125*yhat*zhat;
    Bz[:,23] = @. 0.0625 + -0.125*xhat*zhat + -0.0625*zhat*zhat + 0.125*xhat*zhat;
    Bx[:,24] = @. -0.0625 + -0.0625*zhat + 0.0625*xhat*xhat*zhat + 0.0625*xhat*xhat;
    By[:,24] = @. 0.125 + -0.125*xhat + 0.125*yhat + 0.125*zhat + -0.125*xhat*zhat + -0.125*xhat*yhat*zhat + -0.125*xhat*yhat + 0.125*yhat*zhat;
    Bz[:,24] = @. 0.0625 + 0.125*xhat*zhat + -0.0625*zhat*zhat + -0.125*xhat*zhat;


    B = zeros(3m, 24)
    B[1:m,:] = Bx[1:m,:]
    B[m+1:2*m,:] = By[1:m,:]
    B[2*m+1:3*m,:] = Bz[1:m,:]
    Dhat = zeros(1,24)
    Dhat[1,:] .= 0.125;
    Div = repeat(Dhat, inner=(m,1))
    
    return B, Div
end

GetBasis (generic function with 1 method)

In [5]:
function GetQuadrature3D(Q, quad_mode)
    """ 
    Input:
    Q: number of quadrature points in 1D over [-1,1]
    quad_mode: GAUSS or LOBATTO
    Return:Gauss Quadrature data over [-1,1]^3.
    w: weights of quadrature pts
    qx: quadrature pts in x
    qy: quadrature pts in y
    qz
    """
    
    # 1D Gauss
    if quad_mode == "GAUSS"
        q = zgj(Q, 0.0, 0.0)
        w1 = wgj(q, 0.0, 0.0)
    elseif quad_mode == "LOBATTO"
        q = zglj(Q, 0.0, 0.0)
        w1 = wglj(q, 0.0, 0.0)
    end
    w = zeros(Q*Q*Q)
    qx = zeros(Q*Q*Q)
    qy = zeros(Q*Q*Q)
    qz = zeros(Q*Q*Q)
    for l=1:Q
        for i=1:Q
            for j=1:Q
                k = Q*Q*(l-1) + (i-1)*Q +j
                qx[k] = q[j]
                qy[k] = q[i]
                qz[k] = q[l]
                w[k] = w1[j]*w1[i]*w1[l]
            end
        end
    end
    return w, qx, qy, qz
end

GetQuadrature3D (generic function with 1 method)

# Create rhs qfunction and check with libCEED
$$\int v^T J^T u_e w $$

In [6]:
function VelocityLinear(x, y, z)
    m = length(x)
    ux = @. (x+y+z)
    uy = @. (x-y+z)
    uz = @. (x+y-z)
    
    ue = zeros(3,m)
    ue[1,:] = ux
    ue[2,:] = uy
    ue[3,:] = uz
    return ue
end

function VelocityConstant(x, y, z)
    m = length(x)
    ux = 1
    uy = 2
    uz = 3
    
    ue = zeros(3,m)
    ue[1,:] .= ux
    ue[2,:] .= uy
    ue[3,:] .= uz
    return ue
end

VelocityConstant (generic function with 1 method)

In [7]:
function GetRhs(Coord_E, Q, quad_mode, uexact)
    
    w, qx, qy, qz = GetQuadrature3D(Q, quad_mode)
    # total number of quadrature points
    num_qpts = Q*Q*Q
    # get (x,y,z) on E, detJ,...
    coords, dxdX, detJ = TrilinearMap(Coord_E, qx, qy, qz)
    x = coords[1,:]
    y = coords[2,:]
    z = coords[3,:]
    
    ue = uexact(x,y,z)
    # output of qfunction
    rhs2 = zeros(3*num_qpts,1)
    for i=1:num_qpts
        J = dxdX[:,i,:]
        uu = ue[:,i]
        rhs1 = J' * uu       
        # component 1
        rhs2[i+0*num_qpts, 1] = rhs1[1] *w[i]
        # component 2
        rhs2[i+1*num_qpts, 1] = rhs1[2] *w[i]
        # component 3
        rhs2[i+2*num_qpts, 1] = rhs1[3] *w[i]
    end 
    
    B, Div = GetBasis(qx, qy, qz);
    # ApplyBasis
    rhs = B' * rhs2
    
    return rhs
end

GetRhs (generic function with 1 method)

In [11]:
Q = 3
C_E = [0. 0. 0.;1. 0. 0.;0. 1. 0.;1. 1. 0.;0. 0. 1.;1. 0. 1.;0. 1. 1.;1. 1. 1.]
C_E1 = [0. 0. 0.;0.5 0. 0.;0. 1. 0.;0.5 1. 0.;0. 0. 1.;0.5 0. 1.;0. 1. 1.;0.5 1. 1.]
C_E2 = [0.5 0. 0.;1. 0. 0.;0.5 1. 0.;1. 1. 0.;0.5 0. 1.;1. 0. 1.;0.5 1. 1.;1. 1. 1.]
mode = "GAUSS"
#uexact(x,y,z) = VelocityLinear(x,y,z)
uexact(x,y,z) = VelocityConstant(x,y,z)
rhs = GetRhs(C_E, Q, mode, uexact)

24×1 Matrix{Float64}:
 -1.3333333333333326
 -0.6666666666666663
 -0.33333333333333315
  0.33333333333333315
 -0.33333333333333304
  0.33333333333333304
  0.6666666666666663
  1.3333333333333328
  0.9999999999999991
  1.3333333333333326
  1.6666666666666654
  1.9999999999999987
 -1.3333333333333328
 -1.666666666666666
 -0.33333333333333326
 -0.6666666666666662
 -1.999999999999999
 -1.6666666666666656
 -1.3333333333333326
 -0.9999999999999992
  0.6666666666666663
  0.33333333333333326
  1.666666666666666
  1.3333333333333326

In [12]:
function DiscretizeUe(Coord_E, uexact)
    """
    This function discretize the vector u = uexact(x,
    y,z) on element E
    
    
        local numbering of Hex
    
        5--------7           z
      / |      / |            |
    6 --|---- 8  |            |
    |   |     |  |            /----y
    |   |     |  |           /
    |   1 -------3         x
    | /       | /
    2 --------4
    
    
    """
    # get the normals
    nl, le_l = GetNormal(Coord_E, [0], [-1.],[0],"left")
    nr, le_r = GetNormal(Coord_E, [0], [1.],[0],"right")
    nbt, le_bt = GetNormal(Coord_E, [0.], [0.],[-1.],"bottom")
    nt, le_t = GetNormal(Coord_E, [0.], [0.],[1.],"top")
    nf, le_f = GetNormal(Coord_E, [1.], [0.],[0.],"front")
    nbk, le_bk = GetNormal(Coord_E, [-1.], [0.],[0.],"back")
    
    nbt = nbt*le_bt
    nt = nt*le_t
    nr = nr*le_r
    nl = nl*le_l
    nf = nf*le_f
    nbk = nbk*le_bk

    n1 = Coord_E[1,:]
    n2 = Coord_E[2,:]
    n3 = Coord_E[3,:]
    n4 = Coord_E[4,:]
    n5 = Coord_E[5,:]
    n6 = Coord_E[6,:]
    n7 = Coord_E[7,:]
    n8 = Coord_E[8,:]
    
    nodes = [n1 n3 n5 n7 n2 n4 n6 n8 n5 n6 n7 n8 n2 n1 n6 n5 n1 n2 n3 n4 n4 n3 n8 n7]
    normals = [nbk nbk nbk nbk nf nf nf nf nt nt nt nt nl nl nl nl nbt nbt nbt nbt nr nr nr nr]
    
    x = nodes[1,:]
    y = nodes[2,:]
    z = nodes[3,:]
    uu = uexact(x,y,z)
    ue = uu[1,:] .* normals[1,:] + uu[2,:] .* normals[2,:] + uu[3,:] .* normals[3,:]
    
    return ue
end


DiscretizeUe (generic function with 1 method)

In [13]:
DiscretizeUe(C_E, uexact)

24-element Vector{Float64}:
 -0.25
 -0.25
 -0.25
 -0.25
  0.25
  0.25
  0.25
  0.25
  0.75
  0.75
  0.75
  0.75
 -0.5
 -0.5
 -0.5
 -0.5
 -0.75
 -0.75
 -0.75
 -0.75
  0.5
  0.5
  0.5
  0.5

# Create Local Mass Matrix
$$(v,u_h) = \int \frac{N^T J^T J N w}{detJ} $$

In [26]:
function GetLocalMass(Coord_E, Q, quad_mode)
    
    w, qx, qy, qz = GetQuadrature3D(Q, quad_mode)
    # total number of quadrature points
    num_qpts = Q*Q*Q
    # get (x,y,z) on E, detJ,...
    coords, dxdX, detJ = TrilinearMap(Coord_E, qx, qy, qz)
    
    N, Div = GetBasis(qx, qy, qz);
    # output of qfunction
    tmp = zeros(3*num_qpts,24)
    uu = zeros(3,24)
    for i=1:num_qpts
        J = dxdX[:,i,:]
        uu[1,:] = N[i+0*num_qpts,:]
        uu[2,:] = N[i+1*num_qpts,:]
        uu[3,:] = N[i+2*num_qpts,:]
        #J^T * J * N * w / detJ
        tmp2 = J' * J * uu * w[i] /detJ[i]
        # component 1
        tmp[i+0*num_qpts, :] = tmp2[1,:]
        # component 2
        tmp[i+1*num_qpts, :] = tmp2[2,:]
        # component 3
        tmp[i+2*num_qpts, :] = tmp2[3,:]
    end 
    
    # ApplyBasis
    Me = N' * tmp
    
    return Me
end

GetLocalMass (generic function with 1 method)

In [27]:
Me = GetLocalMass(C_E, Q, mode)

24×24 Matrix{Float64}:
  0.725926     0.285185     0.285185    …  -0.172222    -0.0611111
  0.285185     0.614815     0.148148       -0.00555556   0.105556
  0.285185     0.148148     0.614815       -0.161111     0.0611111
  0.037037     0.285185     0.285185        0.00555556   0.227778
 -0.162963    -0.159259    -0.159259       -0.0611111   -0.172222
 -0.159259    -0.274074    -0.0740741   …   0.105556    -0.00555556
 -0.159259    -0.0740741   -0.274074        0.0611111   -0.161111
 -0.185185    -0.159259    -0.159259        0.227778     0.00555556
  0.00555556  -0.00555556   0.227778       -0.00555556   0.00555556
 -0.161111    -0.172222     0.0611111       0.00555556  -0.00555556
 -0.00555556   0.00555556   0.105556    …   0.105556     0.227778
 -0.172222    -0.161111    -0.0611111       0.227778     0.105556
  0.00555556  -0.161111    -0.00555556     -0.159259    -0.12963
  0.227778     0.0611111    0.105556       -0.12963     -0.159259
 -0.00555556  -0.172222     0.00555556     -

# Test Mass matrix

In [29]:
ue = DiscretizeUe(C_E, uexact)
uh = Me \ rhs
@test isapprox(ue, uh;atol=1e-12)

Test Passed

In [30]:
ue

24-element Vector{Float64}:
  0.0
 -0.25
 -0.25
 -0.5
  0.25
  0.5
  0.5
  0.75
 -0.25
  0.0
  0.0
  0.25
 -0.25
  0.0
 -0.5
 -0.25
  0.0
 -0.25
 -0.25
 -0.5
  0.0
 -0.25
  0.25
  0.0